In [2]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.200  Python-3.11.4 torch-2.1.0+cpu CPU (Intel Core(TM) i5-5200U 2.20GHz)
Setup complete  (4 CPUs, 11.9 GB RAM, 36.8/292.3 GB disk)


In [4]:
model = ultralytics.YOLO('./yolov8n-seg.pt')
result = model.train(
    data="./train_config1.yaml",
    epochs=50,
    # imgsz=640,
    batch=-1
    )

Ultralytics YOLOv8.0.200  Python-3.11.4 torch-2.1.0+cpu CPU (Intel Core(TM) i5-5200U 2.20GHz)
engine\trainer: task=segment, mode=train, model=./yolov8n-seg.pt, data=./train_config1.yaml, epochs=50, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=Fal

In [43]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
model2 = ultralytics.YOLO('./runs/segment/train8/weights/last.pt')
imgsrc = "./image151.jpg"
img = cv2.imread(imgsrc)
results = model2.predict(source=img, conf=0.25)




0: 448x640 1 bear, 92.4ms
Speed: 1.9ms preprocess, 92.4ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)


In [45]:
mask = results[0].masks.xy
overlay = img.copy()
mbox = results[0]

for box in mbox.boxes:
    class_id = mbox. names[box.cls[0].item()]
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    conf = round (box.conf[0].item(), 2)
    display = f' {class_id} {conf}%'
    #ทำนายผล
    image = cv2. putText(img, display, (cords[0], cords[1]-10),cv2. FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    image = cv2. rectangle(image, (cords[0], cords[1]), (cords[2], cords[3]), (255,255,0), 2)
    #วาด Polygon
for i in range(len(mask)):
    x = mask[i].astype (int)
    pts = x. reshape ((-1, 1, 2))
    image = cv2.polylines(image, [pts],isClosed=True, color=(0,0,255), thickness=2)
    overlay = cv2.fillPoly(overlay, pts=[pts], color=(0,0,225))
alpha = 0.4
image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)
r = 600.0 / image.shape[1]
dim = (600, int (image.shape[0] * r))
# perform the actual resizing of the image
resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
cv2.imshow('Marked Image', resized)
cv2.waitKey (0)
cv2.destroyAllWindows ()

2023-10-19 15:04:58.111 Python[12112:303179] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


: 